In [1]:
import geopandas as gpd
from geopandas.geoseries import *

# 447 PLRs
file_path_of_plr = 'berlin-lor.planungsraeume.geojson'
plr = gpd.read_file(file_path_of_plr)

#data.iloc[1].geometry
#data.iloc[1].geometry.area
 
# calc area (for fun ;-)) 
# TODO: investigate if projection is the right one to calc area
# plr.crs
plr['area'] = None
for index, row in plr.iterrows():
    plr.loc[index, 'area'] = row['geometry'].area

plr['bufferedGeometry'] = None
plr['bufferedArea'] = None

# TODO adjust buffer
plr['bufferedGeometry'] = plr.geometry.buffer(0.1)
plr['bufferedArea'] = plr.geometry.area

# add open space to calc which of them intersects with bufferedGeometry 
file_path_of_open_spaces = 'gruenflaechen_shapes.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)
# 2509 open spaces
open_spaces.head()

plr['openSpacesInBufferedPLR'] = np.empty((len(plr), 0)).tolist()
plr['openSpacesInPLR'] = np.empty((len(plr), 0)).tolist()

for indexPLR, rowPLR in plr.head(n=10).iterrows():
    for indexOS, rowOS in open_spaces.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(rowOS.KENNZEICH)
        if rowPLR.geometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(rowOS.KENNZEICH)

newplr = plr


In [2]:
newplr['numberOfSpacesInPLR'] = None
newplr['numberOfSpacesInBufferedPLR'] = None

for index, row in newplr.iterrows():
    newplr.loc[index, 'numberOfSpacesInPLR'] = len(row.openSpacesInPLR)
    newplr.loc[index, 'numberOfSpacesInBufferedPLR'] = len(row.openSpacesInBufferedPLR)


newplr.head()

# TODO
# fläche der Gründflächen (buffered and non) pro PLR
# Menschen pro PLR
# buffer (s.o.)
# n vergrößern (komplette Daten)
# Als Einwohner von PLR x hat man y Grünflächen mit z m^2 Fläche zur Verfügung

# Daten für Waldflächen

,SCHLUESSEL,PLR_NAME,geometry,area,bufferedGeometry,bufferedArea,openSpacesInBufferedPLR,openSpacesInPLR,numberOfSpacesInPLR,numberOfSpacesInBufferedPLR
0,01011101,Stülerstr.,"POLYGON ((13.33889 52.50542, 13.33986 52.50562...",4.85691e-05,"POLYGON ((13.24980 52.46001, 13.24581 52.46886...",0.000049,"[138210, 103600, 159010, 108600, 106210, 14120...","[400000, 242700, 212610]",3,1042
1,01011102,Großer Tiergarten,"POLYGON ((13.33140 52.51288, 13.33141 52.51285...",0.000398559,"POLYGON ((13.23221 52.53046, 13.23251 52.53247...",0.000399,"[138210, 103600, 159010, 108600, 106210, 14120...","[253500, 244010, 210100, 212690, 400000, 24270...",21,1266
2,01011103,Lützowstr.,"POLYGON ((13.34923 52.50307, 13.34972 52.50343...",6.91633e-05,"POLYGON ((13.27489 52.56989, 13.27505 52.57010...",0.000069,"[138210, 103600, 159010, 108600, 106210, 14120...","[216500, 216210, 216400]",3,1114
3,01011104,Körnerstr.,"POLYGON ((13.36254 52.49966, 13.36265 52.49989...",4.47976e-05,"POLYGON ((13.34313 52.40156, 13.33366 52.40392...",0.000045,"[138210, 103600, 159010, 108600, 106210, 14120...","[63361, 216310]",2,1115
4,01011105,Nördl. Landwehrkanal,"POLYGON ((13.35159 52.50976, 13.35199 52.50978...",0.000123753,"POLYGON ((13.25480 52.48490, 13.25302 52.49290...",0.000124,"[138210, 103600, 159010, 108600, 106210, 14120...","[253310, 253400, 211500, 206110, 201900, 20613...",11,1205
